In [1]:
import pandas as pd
import numpy as np

import os
import numpy as np
import seaborn as sns
import matplotlib as plt
import datetime
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
%matplotlib inline

# Load Data

In [2]:
data = pd.read_csv('data/train.csv')
sample_sub = pd.read_csv('data/Sample_sub.csv')
CaptureSite_category = pd.read_csv('data/CaptureSite_category.csv')

## process time
data["Date_TimeCaught"] = pd.to_datetime(data["Date_TimeCaught"])

In [3]:
data.head()

,Rescue_ID,Date_TimeCaught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,Tag_1,...,Lost_Tags,T_Number,CCL_cm,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,ReleaseSite,Date_TimeRelease
0,2000_RE_0060,2000-12-22,Researcher_25,CaptureSite_0,Ocean,Net,Fisher_1072,LandingSite_CaptureSiteCategory_2,Species_6,CC00147,...,NaN,NaN,64.70,62.60,NaN,Unknown,algae at rear of shell,Released,ReleaseSite_50,22/12/00
1,2001_RE_0187,2001-10-28,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_520,LandingSite_CaptureSiteCategory_2,Species_6,W442,...,NaN,NaN,35.85,31.35,NaN,Unknown,multiple b's on front flippers& a lot of alga...,Released,ReleaseSite_62,28/10/01
2,2001_RE_0197,2001-11-01,Researcher_6,CaptureSite_0,Ocean,Net,Fisher_1669,LandingSite_CaptureSiteCategory_2,Species_5,KE0376,...,NaN,NaN,51.80,49.20,NaN,Unknown,clean,Released,ReleaseSite_50,01/11/01
3,2002_RE_0031,2002-03-11,Researcher_32,CaptureSite_0,Ocean,Net,Fisher_1798,LandingSite_CaptureSiteCategory_2,Species_6,CC00302,...,NaN,NaN,60.50,59.00,NaN,Unknown,1 b 3 CS+ calcerous algae at rear end of shell...,Released,ReleaseSite_50,11/03/02
4,2002_RE_0118,2002-08-08,Researcher_25,CaptureSite_0,Ocean,Beached,Fisher_1918,LandingSite_CaptureSiteCategory_2,Species_5,NotTagged_0113,...,NaN,NaN,34.70,33.00,NaN,Unknown,very lively+ right eye is hanging out + swolle...,Released,ReleaseSite_62,08/08/02


In [4]:
data['Date_TimeCaught']

0       2000-12-22
1       2001-10-28
2       2001-11-01
3       2002-03-11
4       2002-08-08
           ...    
18057   2018-12-18
18058   2018-12-18
18059   2018-12-24
18060   2018-12-24
18061   2018-12-28
Name: Date_TimeCaught, Length: 18062, dtype: datetime64[ns]

## Preprocessing

In [8]:
data.shape

(18062, 21)

### Check for Missing days
Check for days not in original dataset from the range of date created

In [5]:
## Get day not in Train day and save as missing_days
train_days = data['Date_TimeCaught'].dt.date.drop_duplicates()
# Create range of days
Main_days = pd.date_range(start=data.Date_TimeCaught.min(),  end=data.Date_TimeCaught.max())
Main_days = pd.DataFrame({'datetime':Main_days.date})
missing_days = Main_days[~Main_days['datetime'].isin(train_days)]

# del train_days, Main_day
missing_days['datetime'] = pd.to_datetime(missing_days.datetime)

/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [10]:
data.Date_TimeCaught.max()

Timestamp('2018-12-31 00:00:00')

### Check for Missing Week
Check for week not in the original dataset

In [6]:
# Create range of weeks
Main_week = pd.date_range("19980417","20190101", freq='W')
Main_week = pd.DataFrame({'datetime':Main_week.date})
Main_week['weekofyear'] =  pd.to_datetime(Main_week['datetime']).dt.weekofyear
# Join year to week
Main_week["year_woy"]= (pd.to_datetime(Main_week['datetime']).dt.year)*100+Main_week['weekofyear']
train_week = ((data['Date_TimeCaught'].dt.year)*100+(data['Date_TimeCaught'].dt.weekofyear)).drop_duplicates()
missing_weeks = Main_week[~Main_week['year_woy'].isin(train_week)]

## Get Target varaible

#### Extract unique_day from data set

In [7]:
all_train_day = data[['Date_TimeCaught','CaptureSite']].sort_values(by='Date_TimeCaught').copy()
# all_train_day["week_year"] = ((data['Date_TimeCaught'].dt.year)*100+(data['Date_TimeCaught'].dt.weekofyear))
unique_train_day = all_train_day.drop_duplicates('Date_TimeCaught')

#### Create unique row for each Capture_site and for each day , get the sum of caught on each site

In [8]:
CaptureSite = list(data.CaptureSite.unique())
## Find away to optimize this code
unique_train_day

for CapId in CaptureSite:
    unique_train_day[str(CapId)] = 0
    events = all_train_day[all_train_day['CaptureSite'] == CapId]
    dates = events['Date_TimeCaught'].unique()
    
    for date in dates:
        number_of_days = len(events[events.Date_TimeCaught == date ])
        unique_train_day.loc[unique_train_day['Date_TimeCaught'].isin([date]), CapId] = number_of_days


/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/zindistars2/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
unique_train_day.drop(['CaptureSite'], axis =1, inplace=True)

/home/zindistars2/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
unique_train_day.head()

,Date_TimeCaught,CaptureSite_0,CaptureSite_1,CaptureSite_10,CaptureSite_11,CaptureSite_12,CaptureSite_13,CaptureSite_14,CaptureSite_15,CaptureSite_16,...,CaptureSite_26,CaptureSite_27,CaptureSite_28,CaptureSite_3,CaptureSite_4,CaptureSite_5,CaptureSite_6,CaptureSite_7,CaptureSite_8,CaptureSite_9
13323,1998-04-17,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1352,1998-07-07,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1353,1998-08-03,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13324,1998-08-07,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1354,1998-09-25,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Do for test

In [11]:
test = pd.date_range('2019-01-01',
                     '2019-12-31',
                           freq="1D")

test = pd.DataFrame({'Date_TimeCaught':test})

# Generate Target
for CapId in CaptureSite:
    test[str(CapId)] = np.NAN


In [12]:
test.head()

,Date_TimeCaught,CaptureSite_0,CaptureSite_1,CaptureSite_10,CaptureSite_11,CaptureSite_12,CaptureSite_13,CaptureSite_14,CaptureSite_15,CaptureSite_16,...,CaptureSite_26,CaptureSite_27,CaptureSite_28,CaptureSite_3,CaptureSite_4,CaptureSite_5,CaptureSite_6,CaptureSite_7,CaptureSite_8,CaptureSite_9
0,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Get target columns
Put all Capture site in one row 

In [14]:
test.set_index('Date_TimeCaught', inplace=True)
unique_train_day.set_index('Date_TimeCaught', inplace=True)

### Convert Caputure_Site to Id

unique_train_day = unique_train_day.reset_index()
test = test.reset_index()

unique_train_day = pd.melt(unique_train_day, id_vars=['Date_TimeCaught'])
unique_train_day.rename(columns={'variable': 'Caputure_Site', 'value': 'target'}, inplace=True)
unique_train_day.set_index('Date_TimeCaught', inplace=True)
# unique_train_day.sort_values('datetime', inplace=True)
unique_train_day = unique_train_day.sort_values('Date_TimeCaught')



test = pd.melt(test, id_vars=['Date_TimeCaught'])
test.rename(columns={'variable': 'Caputure_Site', 'value': 'target'}, inplace=True)
test.set_index('Date_TimeCaught', inplace=True)

test = test.sort_values('Date_TimeCaught')

In [15]:
train = unique_train_day.copy()

In [16]:
train.head()

,Caputure_Site,target
Date_TimeCaught,,
1998-04-17,CaptureSite_0,0
1998-04-17,CaptureSite_22,0
1998-04-17,CaptureSite_23,0
1998-04-17,CaptureSite_1,0
1998-04-17,CaptureSite_13,0


In [17]:
test.head()

,Caputure_Site,target
Date_TimeCaught,,
2019-01-01,CaptureSite_0,NaN
2019-01-01,CaptureSite_15,NaN
2019-01-01,CaptureSite_3,NaN
2019-01-01,CaptureSite_6,NaN
2019-01-01,CaptureSite_12,NaN


# Feature Generation

### Time based Features

In [18]:
attr = ["Hour","Year", 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear','quarter',"weekday_name",
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']

for n in attr: 
    train[n] = getattr(train.index, n.lower())
    test[n] = getattr(test.index, n.lower())

train['Elapsed'] = train.index.astype(np.int64) // 10 ** 9
test['Elapsed'] = test.index.astype(np.int64) // 10 ** 9

In [19]:
## WeekEnd
train['IsWeekend'] = np.where(train['Dayofweek'] < 5,0,1)
test['IsWeekend'] = np.where(test['Dayofweek'] < 5,0,1)

####  Holiday 

In [21]:
#  Holiday 
from pandas.tseries.holiday import *
from pandas.tseries.offsets import CustomBusinessDay

class KenyaHoliday(AbstractHolidayCalendar):
   rules = [
     Holiday('New Year', month=1, day=1, observance=sunday_to_monday),
#      Holiday('Human Rights Day', month=3, day=21, observance=sunday_to_monday),
     Holiday('Good Friday', month=1, day=1, offset=[Easter(), Day(-2)]),
     Holiday('Easter Monday', month=1, day=1, offset=[Easter(), Day(1)]),
#      Holiday('Freedom Day', month=4, day=27, observance=sunday_to_monday),
     Holiday('Workers Day', month=5, day=1,observance=sunday_to_monday),
     Holiday('Madaraka', month=6, day=1, observance=sunday_to_monday),
     Holiday('Huduuma', month=10, day=10, observance=sunday_to_monday),
     Holiday('Masuja', month=10, day=20, observance=sunday_to_monday),
     Holiday('Jumaji Day', month=12, day=12, observance=sunday_to_monday),
     Holiday('Goodwill Day', month=12, day=26, observance=sunday_to_monday),
     Holiday('Christmas', month=12, day=25)
   ]

In [22]:
hol= KenyaHoliday()
myholidays =hol.holidays(start=train.index.min(), end =test.index.max())
hol_day = pd.Series(myholidays)

## Create instance of holiday in a new column
train['IsHoliday'] = np.where((pd.to_datetime(train.index.date)).isin(hol_day) ,1,0)
test['IsHoliday'] = np.where((pd.to_datetime(test.index.date)).isin(hol_day) ,1,0)


# Memory Reduction

In [23]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [24]:
train.shape, test.shape

((151873, 20), (10585, 20))

In [25]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage of dataframe is 23.25 MB
Memory usage after optimization is: 10.65 MB
Decreased by 54.2%
Memory usage of dataframe is 1.58 MB
Memory usage after optimization is: 0.78 MB
Decreased by 50.9%


In [26]:
train = train.sort_values('Date_TimeCaught')
test = test.sort_values('Date_TimeCaught')

## Extra feat

In [27]:
test['ID'] = test['Caputure_Site'].astype(str) + '_' + test['Year'].astype(str) + test['Week'].astype(str).str.zfill(2) 
train['ID']  = train['Caputure_Site'].astype(str) + '_' + train['Year'].astype(str) + train['Week'].astype(str).str.zfill(2) 

### Capture_Site Count

In [28]:
Cap_site_count = data['CaptureSite'].value_counts().reset_index()
Cap_site_count_ratio = data['CaptureSite'].value_counts(True).reset_index()

Cap_site_count=Cap_site_count.rename(columns = {'CaptureSite':'Count',
                                                'index':'Caputure_Site'})

Cap_site_count_ratio=Cap_site_count_ratio.rename(columns = {'CaptureSite':'Count_ratio',
                                                'index':'Caputure_Site'})
# Create week_target(sum of target for each week). 
ID_target = train.groupby('ID')['target'].sum().reset_index()
ID_target=ID_target.rename(columns = {'target':'week_target'})

In [29]:
test['Date_TimeCaught'] = test.index
train['Date_TimeCaught'] = train.index

In [30]:
# Merge Capture_site to train and test data
test = pd.merge(test, Cap_site_count, on=['Caputure_Site'], how="left")
train = pd.merge(train, Cap_site_count, on=['Caputure_Site'], how="left")

test = pd.merge(test, Cap_site_count_ratio, on=['Caputure_Site'], how="left")
train = pd.merge(train, Cap_site_count_ratio, on=['Caputure_Site'], how="left")

train = pd.merge(train, ID_target, on=['ID'], how="left")

In [31]:
test.set_index('Date_TimeCaught', inplace=True)
train.set_index('Date_TimeCaught', inplace=True)

In [32]:
week_train =  train.drop_duplicates('ID')
week_test =  test.drop_duplicates('ID')

## Modelling

In [40]:
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_curve
import matplotlib.pyplot as plt
import gc
import catboost
from catboost import CatBoostClassifier,Pool,cv,CatBoost

In [41]:
del_col = [
#         'Caputure_Site', 
           'target',
           'Hour',
           'Year',
# #            'Month',
# #            'Week', 
#            'Day',
#        'Dayofweek',
        'Dayofyear', 
# #         'quarter', 
        'weekday_name', 
#         'Is_month_end',
#        'Is_month_start',
#         'Is_quarter_end', 
#         'Is_quarter_start',
#         'Is_year_end',
#        'Is_year_start', 
#         'Elapsed',
#         'IsWeekend', 
# #         'IsHoliday', 
        'min',
        'Hour_sin',
       'Hour_cos',
        'ID',
#         'Count',
        'Count_ratio',
    'Fisher',
        'week_target'
        ]

In [35]:
cat_cols = ['Caputure_Site','Month','Week','quarter','Dayofweek','Day']
LE = LabelEncoder()
for col in cat_cols:
    train[col] = LE.fit_transform(train[col])
    test[col] =  LE.transform(test[col])    

# Train Test Split

In [36]:
start      = '2010-01-01'
split_date = '2019-01-01'


val_train = train[(train.index >= start) & (train.index <=split_date)]
val_test = train[train.index >= '2018-01-01']

val_train.shape, val_test.shape

((88189, 24), (9454, 24))

In [42]:
features = [x for x in train.columns if x not in del_col]
features

['Caputure_Site',
 'Month',
 'Week',
 'Day',
 'Dayofweek',
 'quarter',
 'Is_month_end',
 'Is_month_start',
 'Is_quarter_end',
 'Is_quarter_start',
 'Is_year_end',
 'Is_year_start',
 'Elapsed',
 'IsWeekend',
 'IsHoliday',
 'Count']

#### Model Parameters

In [38]:
cat_params= {
            'num_boost_round': 250,
            'use_best_model':True,
            'depth': 2,
            'random_state': 2020,
            'eval_metric': 'RMSE', 
            'loss_function': 'RMSE',
            'verbose': True
}               

CatBoost Training

In [43]:
trn_data = Pool(val_train[features], label=val_train['target'],cat_features = cat_cols)
val_data = Pool(val_test[features], label=val_test['target'],cat_features = cat_cols)


clf = CatBoost(cat_params)
clf.fit(trn_data,verbose_eval = True,eval_set=(val_data))

train_cat = clf.predict(val_train[features])
val_cat = clf.predict(val_test[features])
test_cat = clf.predict(test[features])

# Evalue daily_target
print ('Daily train score', np.sqrt(mean_squared_error(val_train['target'], train_cat)))
print ('Daily test score', np.sqrt(mean_squared_error(val_test['target'], val_cat)))

val_test['pred'] = val_cat
val_train['pred'] = train_cat

# Evalue week_target
val_test_week_pred =  val_test.groupby('ID')['pred'].sum().reset_index()['pred']
val_test_pred_target = val_test.groupby('ID')['target'].sum().reset_index()['target']

val_train_week_pred =  val_train.groupby('ID')['pred'].sum().reset_index()['pred']
val_train_pred_target = val_train.groupby('ID')['target'].sum().reset_index()['target']

print ('Weekly test score', np.sqrt(mean_squared_error(val_test_week_pred, val_test_pred_target)))
print ('Weekly train score', np.sqrt(mean_squared_error(val_train_week_pred, val_train_pred_target)))


Learning rate set to 0.254832
0:	learn: 0.4938323	test: 0.5003755	best: 0.5003755 (0)	total: 106ms	remaining: 26.5s
1:	learn: 0.4866266	test: 0.4919760	best: 0.4919760 (1)	total: 135ms	remaining: 16.7s
2:	learn: 0.4825924	test: 0.4869748	best: 0.4869748 (2)	total: 161ms	remaining: 13.2s
3:	learn: 0.4802620	test: 0.4837583	best: 0.4837583 (3)	total: 184ms	remaining: 11.3s
4:	learn: 0.4787863	test: 0.4821415	best: 0.4821415 (4)	total: 209ms	remaining: 10.2s
5:	learn: 0.4778304	test: 0.4813114	best: 0.4813114 (5)	total: 234ms	remaining: 9.52s
6:	learn: 0.4767034	test: 0.4798456	best: 0.4798456 (6)	total: 255ms	remaining: 8.87s
7:	learn: 0.4760281	test: 0.4788579	best: 0.4788579 (7)	total: 276ms	remaining: 8.35s
8:	learn: 0.4756381	test: 0.4783833	best: 0.4783833 (8)	total: 291ms	remaining: 7.78s
9:	learn: 0.4752995	test: 0.4778380	best: 0.4778380 (9)	total: 304ms	remaining: 7.3s
10:	learn: 0.4750504	test: 0.4776970	best: 0.4776970 (10)	total: 321ms	remaining: 6.98s
11:	learn: 0.4748506	te

95:	learn: 0.4719168	test: 0.4742611	best: 0.4742611 (95)	total: 1.85s	remaining: 2.98s
96:	learn: 0.4719160	test: 0.4742607	best: 0.4742607 (96)	total: 1.87s	remaining: 2.95s
97:	learn: 0.4719004	test: 0.4742605	best: 0.4742605 (97)	total: 1.89s	remaining: 2.92s
98:	learn: 0.4719000	test: 0.4742606	best: 0.4742605 (97)	total: 1.9s	remaining: 2.9s
99:	learn: 0.4718993	test: 0.4742603	best: 0.4742603 (99)	total: 1.92s	remaining: 2.87s
100:	learn: 0.4718975	test: 0.4742603	best: 0.4742603 (100)	total: 1.93s	remaining: 2.85s
101:	learn: 0.4718955	test: 0.4742574	best: 0.4742574 (101)	total: 1.94s	remaining: 2.82s
102:	learn: 0.4718548	test: 0.4743532	best: 0.4742574 (101)	total: 1.97s	remaining: 2.81s
103:	learn: 0.4718541	test: 0.4743534	best: 0.4742574 (101)	total: 1.98s	remaining: 2.78s
104:	learn: 0.4718478	test: 0.4743460	best: 0.4742574 (101)	total: 2s	remaining: 2.76s
105:	learn: 0.4718390	test: 0.4743340	best: 0.4742574 (101)	total: 2.02s	remaining: 2.74s
106:	learn: 0.4718353	tes

194:	learn: 0.4703831	test: 0.4721822	best: 0.4721779 (193)	total: 3.51s	remaining: 991ms
195:	learn: 0.4703643	test: 0.4721771	best: 0.4721771 (195)	total: 3.54s	remaining: 975ms
196:	learn: 0.4703358	test: 0.4721338	best: 0.4721338 (196)	total: 3.55s	remaining: 955ms
197:	learn: 0.4703309	test: 0.4721403	best: 0.4721338 (196)	total: 3.56s	remaining: 936ms
198:	learn: 0.4703142	test: 0.4721503	best: 0.4721338 (196)	total: 3.58s	remaining: 919ms
199:	learn: 0.4703093	test: 0.4721427	best: 0.4721338 (196)	total: 3.6s	remaining: 901ms
200:	learn: 0.4702966	test: 0.4721404	best: 0.4721338 (196)	total: 3.62s	remaining: 884ms
201:	learn: 0.4702956	test: 0.4721434	best: 0.4721338 (196)	total: 3.64s	remaining: 865ms
202:	learn: 0.4702834	test: 0.4721513	best: 0.4721338 (196)	total: 3.66s	remaining: 847ms
203:	learn: 0.4702814	test: 0.4721530	best: 0.4721338 (196)	total: 3.67s	remaining: 828ms
204:	learn: 0.4702813	test: 0.4721536	best: 0.4721338 (196)	total: 3.69s	remaining: 809ms
205:	learn:

/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Prediction

In [46]:
# week_test['Capture_Number'] = test_cat
test['Capture_Number'] = test_cat
# SUm the prediction created by daily_target
final_test = test.groupby('ID')['Capture_Number'].sum().reset_index()

In [47]:
sample_sub = pd.merge(sample_sub[['ID']], final_test, on='ID', how='left')
sample_sub.to_csv('cat_using_day_to_train_others.csv', index=False)

In [48]:
sample_sub

,ID,Capture_Number
0,CaptureSite_0_201901,0.538302
1,CaptureSite_0_201902,0.471588
2,CaptureSite_0_201903,0.453255
3,CaptureSite_0_201904,0.456872
4,CaptureSite_0_201905,0.467531
...,...,...
1271,CaptureSite_9_201940,6.617635
1272,CaptureSite_9_201941,6.809155
1273,CaptureSite_9_201942,6.785604
1274,CaptureSite_9_201943,6.668504
